ChatGPT:tä käytettiin apuna dokumentin rakenteen laatimisessa sekä yksittäisten osien kääntämisessä suomen kielelle. Varsinaiset tekstikuvaukset ja selitykset on kirjoitettu itsenäisesti.

# 1. Business Understanding

Tämän tehtävän tavoitteena on kouluttaa binaarinen luokittelumalli, jonka tarkoituksena on ennustaa numeeristen ominaisuuksien perusteella, jotka on saatu solujen tumista, onko rintakasvain pahanlaatuinen vai hyvänlaatuinen.

Tämä on tärkeää, koska virheelliset negatiiviset tulokset voivat johtaa hoidon viivästymiseen.

# 2. Data Understanding

Tässä osiossa tutustumme aineistoon: lataamme datan, tarkastelemme sarakkeiden ja havaintojen määrää, selvitämme sarakkeiden nimet sekä määrittelemme kohdemuuttujan (target variable / y) ja numeeriset piirteet sen ennustamiseksi (features / X).

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('https://archive.ics.uci.edu/static/public/17/data.csv')
'''df = pd.read_csv('brest.csv')'''
df.head()

,ID,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Diagnosis
0,842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  569 non-null    int64  
 1   radius1             569 non-null    float64
 2   texture1            569 non-null    float64
 3   perimeter1          569 non-null    float64
 4   area1               569 non-null    float64
 5   smoothness1         569 non-null    float64
 6   compactness1        569 non-null    float64
 7   concavity1          569 non-null    float64
 8   concave_points1     569 non-null    float64
 9   symmetry1           569 non-null    float64
 10  fractal_dimension1  569 non-null    float64
 11  radius2             569 non-null    float64
 12  texture2            569 non-null    float64
 13  perimeter2          569 non-null    float64
 14  area2               569 non-null    float64
 15  smoothness2         569 non-null    float64
 16  compactn

In [142]:
cols = []
for col in df.columns:
    cols.append(col)
print(cols)
print(len(cols))

['ID', 'radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1', 'symmetry1', 'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2', 'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3', 'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3', 'symmetry3', 'fractal_dimension3', 'Diagnosis']
32


In [143]:
df.describe()

,ID,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


Aineisto sisältää 32 saraketta (1 ID-sarake, 1 diagnoosisarake ja 30 numeerisia arvoja sisältävää saraketta) sekä 569 riviä.
Diagnoosisarake, jonka arvot ovat M (malignant) ja B (benign), on kohdemuuttuja (target variable / y).
Muut sarakkeet (ID-saraketta lukuun ottamatta) ovat kasvaimen numeerisia piirteitä (features / X), joiden perusteella kohdemuuttujaa ennustetaan.

# 3. Data Preparation

Tässä osiossa valmistelemme aineiston mallin kouluttamista varten: jaamme datan numeerisiin piirteisiin (X) ja kohdemuuttujaan (y), suoritamme aineiston jaon opetus- (train) ja testiaineistoon (test) sekä standardoimme muuttujat ennen mallin kouluttamista.

Lataamme tarvittavat moduulit.

In [144]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Poistamme ID-sarakkeen.

In [145]:
df = df.drop(columns=['ID'])

Muunnamme Diagnosis-sarakkeen numeeriseen muotoon: hyvänlaatuisille arvoille annetaan arvo 0 ja pahanlaatuisille arvo 1.

In [146]:
df['Diagnosis'] = df['Diagnosis'].astype(str)
df['Diagnosis'] = df['Diagnosis'].replace({'B': 0, 'M': 1})
df['Diagnosis'] = df['Diagnosis'].astype('category')

C:\Users\onion\AppData\Local\Temp\ipykernel_22880\1069753751.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Diagnosis'] = df['Diagnosis'].replace({'B': 0, 'M': 1})


Jaamme datan kohdemuuttujaan ja numeerisiin piirteisiin.

In [147]:
X = df.drop(columns=['Diagnosis'])
y = df['Diagnosis']

Seuraavaksi jaamme aineiston opetus- ja testiaineistoon: 70 % käytetään mallin opettamiseen (train) ja 30 % testaamiseen (test).

In [148]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42
)

Koska kNN-algoritmi perustuu euklidisiin etäisyyksiin, standardoimme numeeriset piirteet (features).

In [149]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Näiden vaiheiden jälkeen aineisto on valmis mallin rakentamista ja kouluttamista varten.

# 4. Modeling

Tässä vaiheessa koulutamme kNN-luokittelijan diagnoosin ennustamiseksi ja tutkimme parametrin k vaikutusta mallin laatuun.

Lataamme tarvittavat moduulit.

In [150]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Tässä luomme mallin ja kokeilemme eri k-arvoja löytääksemme tarkimman vaihtoehdon.

In [151]:
k_values = [1, 3, 5, 7, 9, 11, 15]
accuracies = []

for k in k_values:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

Lopuksi tulostamme tulostaulukon.

In [152]:
for i in range(len(k_values)):
    print(f'k = {k_values[i]}, accuracy = {accuracies[i]:.2f}')

k = 1, accuracy = 0.95
k = 3, accuracy = 0.96
k = 5, accuracy = 0.96
k = 7, accuracy = 0.96
k = 9, accuracy = 0.97
k = 11, accuracy = 0.97
k = 15, accuracy = 0.95


Testien perusteella suurin tarkkuus saavutettiin arvoilla k = 9 ja k = 11, joten valitsemme toisen niistä mallin jatkoarviointia varten.

# 5. Evaluation

Tässä vaiheessa arvioimme mallin suorituskykyä testiaineiston avulla, kun malli on koulutettu valitulla parametrin k arvolla (9).

Lataamme tarvittavat moduulit.

In [153]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

Luomme mallin lopullisen version ja koulutamme sen opetusdatalla.

In [154]:
knn_final = KNeighborsClassifier(n_neighbors=9)
knn_final.fit(X_train_scaled, y_train)
y_pred_final = knn_final.predict(X_test_scaled)

Muodostamme valitulle mallille virhematriisin (confusion matrix), joka näyttää testiaineiston oikeiden ja virheellisten luokitusten määrän.

In [155]:
cm = confusion_matrix(y_test, y_pred_final)
cm

array([[106,   2],
       [  3,  60]], dtype=int64)

Tulosten tulkinta:
106 – oikein luokitellut hyvänlaatuiset tapaukset
60 – oikein luokitellut pahanlaatuiset tapaukset
2 – väärät positiiviset
3 – väärät negatiiviset (lääketieteessä vaarallisimmat)

Lopullisen mallin laatumittarit: accuracy, precision ja recall.

In [156]:
acc = accuracy_score(y_test, y_pred_final)
prec = precision_score(y_test, y_pred_final)
rec = recall_score(y_test, y_pred_final)

print(f'Accuracy: {acc:.2f}')
print(f'Precision: {prec:.2f}')
print(f'Recall {rec:.2f}')


Accuracy: 0.97
Precision: 0.97
Recall 0.95


Saadut tulokset osoittavat, että malli luokittelee suurimman osan havainnoista oikein.
Tarkkuus (accuracy) on 0.97, mikä tarkoittaa, että 97 % diagnooseista ennustettiin oikein.
Recall-arvo on 0.95, mikä tarkoittaa, että malli tunnistaa suurimman osan pahanlaatuisista kasvaimista, mutta joissakin tapauksissa virheitä voi silti esiintyä.
Precision-arvo on 0.97, mikä tarkoittaa, että suurin osa pahanlaatuisiksi ennustetuista kasvaimista on todellisuudessa pahanlaatuisia.

# 6. Deployment

Tässä työssä rakennettiin kNN-algoritmiin perustuva malli kasvaimen diagnoosin (pahanlaatuinen tai hyvänlaatuinen) ennustamiseksi numeeristen piirteiden avulla.

Mallinnusvaiheessa testattiin useita parametrin k arvoja, ja optimaaliseksi valittiin arvo k = 9.

Testiaineistolla malli saavutti hyvät tulokset: accuracy = 0.97, precision = 0.97 ja recall = 0.95. Tämä osoittaa, että malli pystyy useimmissa tapauksissa määrittämään diagnoosin oikein ja tunnistamaan pahanlaatuiset kasvaimet tehokkaasti.

Tästä huolimatta mallilla on rajoituksia, sillä se on koulutettu ja testattu vain yhdellä aineistolla. Tulosten luotettavuuden parantamiseksi voidaan jatkossa käyttää k-kertaisen ristiinvalidoinnin (cross-validation) menetelmää tai testata muita luokittelualgoritmeja.

Saatua mallia voidaan käyttää ainoastaan päätöksenteon tukena, eikä sitä tule pitää itsenäisenä lääketieteellisenä diagnostiikkajärjestelmänä.